# Pre steps before creating Datalab Notebook
As this needs a GCP Project below steps were perfomed to create Environment for Development
#Pre steps before creating Datalab VM
* Signed up for GCP Trial with new e-mail address
* Install Google cloud SDK tools on my machine
* Open Google cloud SDK shell CLI
* Cross check Gcloud config - gcloud config list
* If not desired project and admin user account, who have permissions create resources - change this by running below commands
* Run this command to change project - gloud config set project project_id 
* Run this command to change account - gcloud config set account user_account_name
* Install Datalab components - gcloud components install datalab
* Created Datalab instance named "AICoinMarketCap" - datalab create --zone us-east1-b AI-CoinMarketCap
* If any authorization error comes then go to link provided and authorize to access the Project to give access to project.
* Gcloud auth login – allowed access to Gcloud SDK
* Created Datalab instance named "AICoinMarketCap" - datalab create --zone us-east1-b AI-CoinMarketCap
* Go to http://localhost:8081/ as given on console
* create a new Notebook CoinMarketCap_DataAnalytics_Solution



# Create Storage bucket and BQ Dataset using API

Check the Project context to make sure that we are using the correct Project

In [165]:
from google.datalab import Context

project = Context.default().project_id
print (project)


casertachallenge


Create bucket using storage API to store downloaded data files at this location

In [166]:
import google.datalab.storage as storage

cmcd_bucket_name = 'coin_market_cap_data_files'
bucket = storage.Bucket(cmcd_bucket_name)
if(bucket.exists() <> True) :
  bucket.create()
  print cmcd_bucket_name, " bucket is created" 
else :
    print "Bucket already exists, name: ", cmcd_bucket_name


Bucket already exists, name:  coin_market_cap_data_files


Create BigQuery dataset using bigquery API to make data available for further analysis by using SQL syntax

In [168]:
import google.datalab.bigquery as bq

cmcd_dataset_name = 'coin_market_cap_data'
cmcd_dataset = bq.Dataset(cmcd_dataset_name)

if(cmcd_dataset.exists() <> True) :
  cmcd_dataset.create()
  print cmcd_dataset_name, " Dataset is created" 
else :
    print "Dataset already exists, name: ", cmcd_dataset_name

coin_market_cap_data  Dataset is created


# Data ingestion script to Coin Market Cap Data

CoinMarketAPI provid public API to download the data https://coinmarketcap.com/api/
Summary of this API
* 	Listings (https://api.coinmarketcap.com/v2/listings/) – List all Bitcoin stock symbols. No parameters
*	Ticker (https://api.coinmarketcap.com/v2/ticker/) – Fetches all stock ticker crrent data with limit 100 rows by Rank.
    Parameters - start, limit, convert(value USD gives all the currencies in US Dollar amount)
*	Ticker for Specific coin (https://api.coinmarketcap.com/v2/ticker/{id}) – Fetches stock ticker changes of specified id (No limit was mentioned; Assumption is we can get all data with one call). Listings API gives all ids data, then this endpoint can be called for specific coin data.
*	Global (https://api.coinmarketcap.com/v2/global/) – Global Market data (Gives summary). Parameters - convert(value USD gives all the currencies in US Dollar amount)

Report Queries need data of all tickers with current chages with current price, market cap, supply, growth pecetages. All this data can be fetched using Ticker endpoint. And cost information must be USD (Most Likely Euro) - Defualt price was not in USD so use parameters and fetch all coin data in USD. And Ticker providing values in order of Rank, which might change every day but this does not matter as we iterate through get all the values. 

We can follow another method, by pulling Listings data first and then ticker data by coin id. This will result more number of calls so choosing earlier method as it suffices our Reporting needs.


** Clean up script **
Run this only when you would like to want to debug/ execute everything again.

In [149]:
# Delete all files from Bucket
# Only for debugging purposes

import google.datalab.storage as storage

cmcd_bucket_name = 'coin_market_cap_data_files'
bucket = storage.Bucket(cmcd_bucket_name)
objectslist = storage.Objects(cmcd_bucket_name, "", ',', ) 

for eachobject in objectslist: 
  eachobject.delete()
  
print 'All blobs in Bucket \'{0}\' are deleted'.format(cmcd_bucket_name)

All blobs in Bucket 'coin_market_cap_data_files' are deleted


**Data Ingeston Script** #start

* As we would like to keep number of API endpoint calls to minimum. 
* This script will make calls to Ticker endpoint with parameters -- start, limit, convert(value USD)
* Downloads Json part files and upload them to Storage bucket
* At the end uploads combined csv file to Storage bucket

**Steps**
* Get total number of Crypto currencies in the Market
* Iterate and get the Ticker data for all currencies
* Save the json object to Bucket
* Flatten json object and keep appending to Data frame
* After getting all data, upload the csv to Bucket as well

In [182]:
# Download all files to Bucket
import json
import pandas as pd
import google.datalab.storage as storage

cmcd_bucket_name = 'coin_market_cap_data_files'
bucket = storage.Bucket(cmcd_bucket_name)

# fetch parameter variables
var_start= 1
var_limit = 1
var_convert = 'USD'
var_blob_json_name_static = 'cmc_ticker_data_details_partxxxx.json'
var_json_content_type = "application/json"
var_blob_csv_name_static = 'cmc_ticker_data_details.csv'
var_csv_content_type = "application/csv"
var_file_num = 1
var_fill_length = 4
var_total_coins = 1 # Get actual value from API

# get total number of coins to iterate
fetch_parameters = { "start": var_start, "limit": var_limit, "convert": var_convert }
response = requests.get("https://api.coinmarketcap.com/v2/ticker/", params=fetch_parameters)
data = response.json()
var_total_coins = data["metadata"]["num_cryptocurrencies"]
# print(response.content) -- see what above request returning, top 1 object with metadata.
# print(data["metadata"]["num_cryptocurrencies"]) -- Returns 1614(Total number of coins)

var_limit = 100
var_fill_length = len(str(var_total_coins/var_limit))
print "Total coins", var_total_coins

converted_csv_df = pd.DataFrame()

# Iterate to get all Ticker data of all coins
while(var_start <= var_total_coins):
  print "Start", var_start, "fetched with limit till", var_start+var_limit-1
  fetch_parameters = { "start": var_start, "limit": var_limit, "convert": var_convert }
  response_itr = requests.get("https://api.coinmarketcap.com/v2/ticker/", params=fetch_parameters)
  data_itr = response_itr.json()
    
  response_data_object = data_itr["data"]
  for objectss in response_data_object:
    child_objects = json.loads(objectss)
    flattened_coin_ticker_data = pd.io.json.json_normalize(response_data_object[str(child_objects)])
    converted_csv_df = converted_csv_df.append(flattened_coin_ticker_data)
  
  var_blob_json_name_part = var_blob_json_name_static.replace('xxxx', str(var_file_num).zfill(var_fill_length))
  blob_file_ref = bucket.object(var_blob_json_name_part)
  blob_file_ref.write_stream(response_itr.content, var_json_content_type)
  #print var_blob_json_name_part, " uploaded"
  var_start = var_start + var_limit
  var_file_num = var_file_num + 1

converted_csv_df.columns = converted_csv_df.columns.str.replace(r"[.]", "_")
blob_file_ref = bucket.object(var_blob_csv_name_static)
blob_file_ref.write_stream(converted_csv_df.to_csv(None, ',', '', None, None, True, False, "index", 'w', 'UTF-8') , var_csv_content_type)

print ''''''
print 'Number of json parted files generated: ', (var_file_num - 1), 'files named as ', var_blob_json_name_static 
print 'All json files uploaded to Bucket: ', cmcd_bucket_name
print ''''''
print 'Combined CSV file generated and uploaded to Bucket: ', cmcd_bucket_name, 'file name: ', var_blob_csv_name_static
# Get the response data as a python object.  Verify that it's a dictionary.
#data = response.json()
#print(type(data))
#print(data)
#print(response.headers)

Total coins 1614
Start 1 fetched with limit till 100
Start 101 fetched with limit till 200
Start 201 fetched with limit till 300
Start 301 fetched with limit till 400
Start 401 fetched with limit till 500
Start 501 fetched with limit till 600
Start 601 fetched with limit till 700
Start 701 fetched with limit till 800
Start 801 fetched with limit till 900
Start 901 fetched with limit till 1000
Start 1001 fetched with limit till 1100
Start 1101 fetched with limit till 1200
Start 1201 fetched with limit till 1300
Start 1301 fetched with limit till 1400
Start 1401 fetched with limit till 1500
Start 1501 fetched with limit till 1600
Start 1601 fetched with limit till 1700

Number of json parted files generated:  17 files named as  cmc_ticker_data_details_partxxxx.json
All json files uploaded to Bucket:  coin_market_cap_data_files

Combined CSV file generated and uploaded to Bucket:  coin_market_cap_data_files file name:  cmc_ticker_data_details.csv


**Data Ingeston Script** #end

# Upload csv file to BigQuery Dataset

In [206]:
import google.datalab.bigquery as bigquery
import google.datalab.storage as storage
from google.datalab import Context
import pandas as pd
from io import BytesIO

#Bucket reference
cmcd_bucket_name = 'coin_market_cap_data_files'
cmcd_data_file_pattern = 'cmc_ticker_data_details.csv'
var_bucket_prefix = "gs://"
var_source_uri = (var_bucket_prefix + cmcd_bucket_name + '/' + cmcd_data_file_pattern)

# Create table References
project_id = Context.default().project_id
cms_data_set_id = "coin_market_cap_data"
cmc_ticker_data_table_id = "cmc_ticker_data_details"
cmc_ticker_data_details_tbl_bq_name = bigquery.TableName(project_id, cms_data_set_id, cmc_ticker_data_table_id, None)
cmc_ticker_data_details_bq_tbl = bigquery.Table(cmc_ticker_data_details_tbl_bq_name)

# Create table if does not exists
if(cmc_ticker_data_details_bq_tbl.exists() <> True):
  csv_file_df = pd.read_csv(BytesIO(storage.Object(cmcd_bucket_name, "cmc_ticker_data_details.csv").download()), sep = ',')
  cmc_ticker_data_details_tbl_bq_schema = bigquery.Schema.from_data(csv_file_df.head())
  #print cmc_ticker_data_details_tbl_bq_schema
  cmc_ticker_data_details_bq_tbl.create(cmc_ticker_data_details_tbl_bq_schema, False)

# Options to load data into table
var_csv_delimiter = ','
var_csv_skip_leading_rows = 1
var_csv_encoding = 'utf-8'
var_csv_quote = '"'
var_csv_allow_quoted_newlines = False
var_csvallow_jagged_rows =  False
load_csv_options = bigquery.CSVOptions(var_csv_delimiter, var_csv_skip_leading_rows, var_csv_encoding, var_csv_quote, var_csv_allow_quoted_newlines, var_csvallow_jagged_rows)
var_ignore_unknown_values = False
var_max_bad_records = 0
var_load_mode = 'overwrite'
var_source_format = 'csv'

cmc_ticker_data_details_bq_tbl.load(var_source_uri, var_load_mode, var_source_format, load_csv_options, var_ignore_unknown_values, var_max_bad_records)

print "Coin Market cap Data load into BQ table '{0}' is completed".format(cmc_ticker_data_table_id)


Coin Market cap Data load into BQ table 'cmc_ticker_data_details' is completed


# Query the Data from Big Query table

Queries to be Executed
* How many coins have a USD price greater than 8000 Dollars?
* What is the total market cap of the top 100 cryptocurrencies (in USD)?
* Which coins have an available supply less than 5M Dollars?
* Which 5 coins have seen the greatest percentage growth in the last week?
* How many ticker symbols contain the letter "X" ?


In [208]:
%%bq query --name read_columns
select * from `coin_market_cap_data.cmc_ticker_data_details` Limit 10

In [211]:
read_columns.execute().result()

circulating_supply,id,last_updated,max_supply,name,quotes_USD_market_cap,quotes_USD_percent_change_1h,quotes_USD_percent_change_24h,quotes_USD_percent_change_7d,quotes_USD_price,quotes_USD_volume_24h,rank,symbol,total_supply,website_slug
106019270.0,1421,1525602548.0,,JobsCoin,30432.0,,-1.98,55.1,0.000287045,,1295,JOBS,200019300.0,jobscoin
560000010.0,2694,1525540164.0,,Nexo,133042562.0,,,,0.237576,,1014,NEXO,1000000000.0,nexo
813229.0,1228,1525440248.0,5000000.0,Enigma,112413.0,,,,0.13823,,1223,XNG,1225570.0,enigma
390312753.0,833,1525671844.0,,GridCoin,19779021.0,-0.18,-5.1,-6.43,0.0506748,27752.3,402,GRC,390312753.0,gridcoin
24101381.0,671,1525671844.0,,GAIA,592949.0,-0.18,0.81,12.46,0.0246023,20.7324,1088,GAIA,24101381.0,gaia
20343100.0,656,1525671844.0,,Prime-XI,265410.0,-0.18,-4.57,32.23,0.0130467,526.124,957,PXI,20343100.0,prime-xi
19695124.0,654,1525671844.0,,DigitalPrice,2176823.0,-7.11,0.95,-1.15,0.110526,10579.4,770,DP,27195125.0,digitalprice
40710246.0,588,1525671844.0,,Ubiq,89968422.0,-0.43,-5.64,-0.23,2.20997,292817.0,179,UBQ,40710246.0,ubiq
4571021.0,633,1525671844.0,,ExclusiveCoin,5579343.0,-0.1,-4.91,-6.79,1.22059,66547.0,641,EXCL,4571021.0,exclusivecoin
573276226.0,638,1525671844.0,,Trollcoin,645016.0,1.74,1.25,-5.62,0.00112514,440.688,1085,TROLL,573276226.0,trollcoin


In [233]:
%%bq query --name count_price_gt_8000
Select count(1) count_price_gt_8000 from `coin_market_cap_data.cmc_ticker_data_details`
where quotes_USD_price > 8000


In [234]:
count_price_gt_8000.execute().result()

count_price_gt_8000
5


In [246]:
%%bq query --name coins_price_gt_8000
Select symbol, name, quotes_USD_price from `coin_market_cap_data.cmc_ticker_data_details`
where quotes_USD_price > 8000
order by quotes_USD_price DESC

In [247]:
coins_price_gt_8000.execute().result()

symbol,name,quotes_USD_price
BTWTY,Bit20,871427.0
NANOX,Project-X,121148.0
42,42-coin,35059.6
BITBTC,bitBTC,11547.5
BTC,Bitcoin,9348.23


In [248]:
%%bq query --name total_market_cap_top_100
Select (sum(quotes_USD_market_cap)) total_market_cap_top_100 from `coin_market_cap_data.cmc_ticker_data_details`
where rank <= 100

In [249]:
total_market_cap_top_100.execute().result()

total_market_cap_top_100
4.13556281883e+11


Here we have three supply number (Circulating_suppply, max_supply, total_supply), In this total_supply is considered available supply as this designates as available for people to exchange/ transact with. Others are max_supply is that maximum that can be with its crypto currency algorithm, which can be mined. and circulating_supply is something that is in market and being exchanged in the Markets. So writing Query to pull this.

In [250]:
%%bq query --name available_supply_gt_5M
Select symbol, name, total_supply from `coin_market_cap_data.cmc_ticker_data_details`
Where total_supply < 5000000
order by total_supply ASC

In [251]:
available_supply_gt_5M.execute().result()

symbol,name,total_supply
NANOX,Project-X,1.0
BTWTY,Bit20,1.0
42,42-coin,42.0
BITBTC,bitBTC,51.0
BITGOLD,bitGold,189.0
300,300 Token,300.0
PBT,Primalbase Token,1250.0
IDXM,IDEX Membership,2000.0
RSGP,RSGPcoin,2160.0
KNC,KingN Coin,2509.0


In [254]:
%%bq query --name highest_growth_past_week
Select symbol, name, quotes_USD_percent_change_7d from `coin_market_cap_data.cmc_ticker_data_details`
order by quotes_USD_percent_change_7d DESC limit 5

In [255]:
highest_growth_past_week.execute().result()

symbol,name,quotes_USD_percent_change_7d
NOTE,DNotes,22240.7
UCASH,U.CASH,263.82
TCOIN,T-coin,187.54
XCN,Cryptonite,177.09
SOC,All Sports,174.38


In [260]:
%%bq query --name symbol_with_X
Select symbol, name from `coin_market_cap_data.cmc_ticker_data_details`
where REGEXP_CONTAINS(symbol, r'[x|X]')

In [261]:
symbol_with_X.execute().result()

symbol,name
OX,OX Fina
PX,PX
TX,TransferCoin
X2,X2
XP,Experience Points
ADX,AdEx
AIX,Aigang
APX,APX
ATX,Artex Coin
AXP,aXpire


Referred to 
https://google-cloud-python.readthedocs.io/en/latest/storage/blobs.html
http://googledatalab.github.io/pydatalab/google.datalab.storage.html
